<h1 style = "font-family: avenir"><em>A Higher Revelation: An Algorithmic Predictor of User Music Preferences</em></h1>
<h3 style = "font-family: avenir">By Kyle Vu</h3>
<h4 style = "font-family: avenir">August 6, 2018</h4>

![Tame Impala performing at Lollapalooza 2015](images/Tame-Impala-Lollapalooza-Berlin-2015-billboard-650.jpg)
<center><h5>Image Source: Frank Hoensch</h5></center>

>**"I must despise the world which does not know 
>that music is a higher revelation than all wisdom and philosophy."**
>- *Ludwig van Beethoven (allegedly)*

Entering college, I thought that I might declare a major in philosophy. This pursuit concluded when I realized that I viewed philosophy only as a means to garner hipster cred by quoting Nietzsche conversationally. Even after abandoning the discipline of philosophy, I still managed to achieve minor enlightenment of another kind- college was the first time that I began listening to classical music. Classical music was a new frontier for me. I'd long viewed the genre as too exclusive, too pretentious, too lifeless to justify a listen. Then, during an episode of procrastination, I stumbled upon a recording of the [Chicago Symphony Orchestra performing Beethoven's final completed symphony.](https://www.youtube.com/watch?v=rOjHhS5MtvA) 

I was entranced. I could observe in the CSO video all of the qualities that make music so captivating to me: the synchrony of so many moving parts, the rock star mane of Riccardo Muti, and the transcendent exposition of emotion from the performers. Formerly inclined never to venture beyond my safe haven genres of hip-hop and pop, I began to listen to more classical, more jazz, more anything. I even dabbled into those solitary genres that I had previously embargoed from my music library: country and metal.

As my music taste has expanded, music itself has taken up a more substantial mantle in my life. I relish the opportunity to chat about music recommendations with friends and have started to make music myself (I began to teach myself how to play the ukulele and piano within the past few years). At this point, it almost goes without saying that music is persistently on my mind. Nowadays, as my appreciation for data science grows in parallel with my appreciation for music, I think a lot about how to find good music. Listening to bad music bears some measure of disutility for me: those are 2-5 minutes of my life that I'll never get back. I began thinking about how I could find good music (relative to my standards) more efficiently. Once I discovered that Spotify offers an API for developers, I wanted to explore the application of machine learning algorithms to this question of music taste.

<h2 style = "font-family: avenir">Getting the Data</h2>

To acquire the desired information about my music taste, I used the cleverly-named [Spotipy](https://github.com/plamere/spotipy) library to interact with the Spotify API. After authenticating and creating a Spotify API object, sailing was relatively smooth. All that is required to grab data about a playlist is your Spotify user name and that playlist's unique ID. 

The only hiccup arose from a built-in parameter of the `user_playlist_tracks` method, which only allows users to grab data for a maximum of 100 tracks. After defining a function to overcome this, I was able to receive a dictionary object containing all 1,128 tracks that I sought to analyze. This playlist object included 707 tracks that I enjoyed and 421 tracks that I disliked.

In [ ]:
# create function to work around 100-song limit
def get_playlist_tracks(name, playlist_id):
    results = spot.user_playlist_tracks(name, playlist_id)
    playlist = results['items']
    while results['next']:
        results = spot.next(results)
        playlist.extend(results['items'])
    return playlist

With this dictionary object in tow, I simply had to loop through some keys to accumulate each track's title, artist, and unique ID. I used the latter to compile a list of each track's audio features, which are easily accessed with the `audio_features` method.

In [ ]:
# define function to get data frame from playlist dict.
def make_song_frame(playlist):
    id_lst = []
    title_lst = []
    artist_lst = []
    for i in range(len(playlist)):
        id_lst.append(playlist[i]['track']['id'])
        title_lst.append(playlist[i]['track']['name'])
        artist_lst.append(playlist[i]['track']['artists'][0]['name'])

    features_lst = []
    for song_id in id_lst:
        audio_features = spot.audio_features(song_id)
        features_lst.append(audio_features)

    features_lst = list(chain.from_iterable(features_lst)) # unnest features
    title_frame = pandas.DataFrame(title_lst)
    artist_frame = pandas.DataFrame(artist_lst)
    frivolity_frame = pandas.concat([title_frame, artist_frame], axis = 1)
    feature_frame = pandas.DataFrame(features_lst)
    song_frame = pandas.concat([frivolity_frame, feature_frame], axis = 1)

    return song_frame

Linking all of this information together into one data frame, the data was now ready for analysis.

<h2 style = "font-family: avenir">Exploring the Data</h2>

The expedition through the Spotify API returned a 1128x16 data frame. For a given track in my data set, I had gathered the following information: the track's title, artist, acousticness, danceability, energy, instrumentalness, key, liveness, loudness, mode, speechiness, tempo, time signature, valence, duration in seconds, and an indicator of whether I liked or disliked the song. Navigate [here](https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/) for further information about these audio qualities.

To explore relationships among the 14 variables (the title and artist variables were not applied in modeling), I plotted a correlogram.

![Correlogram](images/correlogram.png)

Reading off the top row of the correlogram suggests that none of the predictors has a very strong linear relationship with the dependent variable `like`. Indeed, there are few salient pairwise linear relationships displayed within the matrix. Let's take a look at some of the strongest relationships portrayed in the correlogram:

1. `energy` and `loudness` are very positively correlated with each other. Cranky neighbors all over the globe nod in agreement.
2. `acousticness` is negatively correlated with `loudness` and `energy`. This seems sensible. When I think of acoustic music, I think of relaxed open mic performances and undergrads impressing their friends by strumming "Wonderwall" on their guitars.
3. `valence` is positively correlated with `danceability` and `energy`. `valence` is best interpreted as a song's "mood": high values are reserved for very cheerful tracks, low values for more somber ones.

After honing in on these four specific variables of interest, I rendered scatterplots to further explore these relationships. 

![Faceted Scatterplot](images/facet_scatter.png)

Synthesizing the conclusions gleaned from the three subplots, it seems that I am more likely to enjoy tracks given that they are quieter, more acoustic, less cheerful, and lower energy. This rule is not perfectly deterministic, of course- the patches of blue bruising amongst the red splotches across the scatterplots attest to that. In light of this fact, a useful heuristic still arises from this analyis: I have the music taste of a curmudgeon.

Careful observers might notice a lone purple drop of purple floating within a sea of red and blue. This point maps ["Mia & Sebastian's Theme,"](https://www.youtube.com/watch?v=8zscp5a7gk0) the all-time most played track in my iTunes library. Cultured observers might recall that this is the best composition of the young millenium. I enjoy this song so much that it motivated me to start learning to play piano during my sophomore year of college.

<h2 style = "font-family: avenir">Applying Machine Learning Models</h2>

Predicting whether I will enjoy or dislike a given track is a question of classification, so I opted to fit the following models on the dataset: logistic regression, K-nearest neighbors, decision tree, and random forest. All but the first require some hyperparameter tuning to optimize misclassification error:

<img alt "Hyperparameter Tuning" src = "images/tuning.png">

After applying 10-fold cross validation to tune the hyperparameters, I trained each model on 80% of the dataset (902 tracks). The remaining 20% (226 observations) was be reserved for testing.

All four models that I ran performed better than a coin toss would have- predictive performance ranged from K-nearest neighbors' 59.3% accuracy on the test set to random forest's 65.9%. Though predictive accuracy is obviously a useful performance metric, it doesn't tell the whole story. I was also interested in the predictions that the models got *wrong*. Were the models prone to false positives or false negatives? Or are the errors they make evenly distributed between Type I and Type II? I constructed the confusion matrices below to learn more about the mistakes made by each model.

<center><img alt = "K-Nearest Neighbors Confusion Matrix" src = "images/knn.png" style = "display: inline; width: 400px; height: 320px;">
<img alt = "Random Forest Confusion Matrix" src = "images/forest.png" style = "display: inline; width: 400px; height: 320px;"></center>

<center><img alt = "Logistic Regression Confusion Matrix" src = "images/log.png" style = "display: inline; width: 400px; height: 320px;">
<img alt = "Decision Tree Confusion Matrix" src = "images/tree.png" style = "display: inline; width: 400px; height: 320px;"></center>

Given that an incorrect prediction was made, Type I errors were more common for the logistic regression and random forest models. Type II errors were more prevalent for the K-nearest neighbors and decision tree models. In the case of false positives, the model would present users with tracks that they end up skipping over. In the case of false negatives, the model would neglect to present users with tracks that might end up being beloved gems. Which is the lesser of two evils here? 

The answer might depend on the level of subscription that a given user has. Those without Spotify Premium are constrained in the number of skips they can use, while their Premium counterparts are not. From my own experience as a Premium user, I might be inclined to favor the models more prone to Type I errors- I'd rather simply skip songs that I don't enjoy, given that I don't miss out on those hidden gems that the model would recommend. Skipping a song presents some degree of opportunity cost, but I would personally be willing to spend 10-30 seconds listening to a bad track before skipping it if meant I could eventually discover quality tracks that I otherwise wouldn't have encountered.

<h2 style = "font-family: avenir">Moving Forward</h2>

A quick comment about the data: it's imperfect. The Spotify API does a wonderful job of providing users with clean data that requires little feature engineering. The primary shortcoming of this dataset was the methodology I used to compile the playlists of songs I like and songs I dislike. The corpus of 1,128 songs used in this analysis is far from comprehensive. The `Like` playlist comprises only those songs that I saved while using Spotify Premium, a period spanning a little over a year. Some of the music that I enjoy is housed in my iTunes library and has not been completely migrated to Spotify. They are, therefore, excluded from the analysis. Even while I write this sentence, I can recall cherished songs that were left out of the dataset ([Tina Turner's "Proud Mary"](https://www.youtube.com/watch?v=02QUmKVsyFYs) comes to mind). Additionally, not all my beloved songs are even in the Spotify catalog. For example, Beyoncé's *Lemonade* is [notoriously excluded from the streaming service's offerings.](https://qz.com/670521/spotify-has-one-thing-to-say-about-its-lack-of-lemonade/)

The `Dislike` playlist was compiled on an even more *ad hoc* basis than the `Like` playlist- I conceived of this project, then tried to recall as many reviled songs as I could. The former playlist ended up being more biased than I would have liked for several reasons:

1. the fact that the construction of the `Dislike` playlist was so reliant on my own recollection that availability bias undoubtedly tints the data
2. some of the songs in the `Dislike` playlist were added after I thought of artists whose music I found polarizing (e.g. Maroon 5) and partitioned their discographies into `Like` and `Dislike,` where appropriate
3. similar to the case with the `Like` playlist, the `Dislike` playlist hardly encompasses all of the songs that I hate

The ideal way to gather data for this project would be to begin logging from birth all of the songs that I've listened to throughout my two decades of living and tag each song with a binary 'yes' or 'no.' Unfortunately, I lacked the foresight to do so as a child.

In the future, I'd like to apply some of the methods presented here to data that more comprehensively captures my music tastes and to data sourced from peers. Perhaps this approach will enable quantitative comparisons of peer-to-peer music tastes. Any attempt at such comparisons would require that all participants compile a playlist of songs that they hate (I've yet to meet anybody who keeps around a dedicated playlist of disliked tracks). 

Even without such a playlist, some neat comparisons can still be made. My friend Guin Mesh was kind enough to allow me to use her playlist for this analysis. With over 1,200 songs in one playlist, Guin has certainly built up a more robust population of liked tracks in Spotify than I have so far. Looking at the medley of navy (my favorite color), green (Guin's favorite color, I think?), and orange, some intriguing patterns arise. Notice in the bottom subplot that I appear to be more amenable to very quiet songs than Guin is. Looking at the tabulation of mutually liked songs reveals that Guin and I both love Tame Impala and Frank Ocean- songs by these artists comprise 16 out of the 77 tracks that our playlists share.

![Scatterplot Comparing Friends' Music Tastes](images/peer.png)

In addition to acquiring a larger dataset through interaction with peers' playlists, I'd also like to implement other modeling tools (e.g. Keras neural network or gradient boosting) to further improve predictive power. With that in mind, this exercise was still incredibly fun for me. I'm especially grateful to Spotify for creating an API that grants users access to such quality data. Without them, I wouldn't have been able to separate the signal from the noise.

<strong><em>Check out [the code for this project!](https://github.com/kvu1/spot-a-pie)</em></strong>